```python
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
        with torch.no_grad():
            tot_loss,tot_acc,count = 0.,0.,0
            for xb,yb in valid_dl:
                pred = model(xb)
                n = len(xb)
                count += n
                tot_loss += loss_func(pred,yb).item()*n
                tot_acc  += accuracy (pred,yb).item()*n
        print(epoch, tot_loss/count, tot_acc/count)
    return tot_loss/count, tot_acc/count
```

Above is a minimalist training loop. It does couple of things as below:
- Training through a number of epochs
- In each epoch, loop through every batchs
- In each batch, depending on if the model is training or validating, having different behaviours

However, in most of the time, we won't stop here after finishing the 1st experiment. What if ... :
- The dataset is unbalance, we should better adding more metrics than just accuracy, maybe F1, ROCAUC
- You want to log the result in a file / show in a realtime graph put it on WandB
- Early Stopping, Save Best Model, and Much more ...

If we will just adding more and more things in the training loop, it will eventually become a big mess with thoudsand of lines, unmaintainable. And if you want to disable some features, then enable it again, there might be a big chance that you will just create new notebook with Learner-Copy1, Learner-Copy2, Learner-CopyN, ...

Let's today design a new Flexible Training Loop where we keep it as simple as possible but also having a full power to adding things via callbacks. 

Callback here is a not a python feature but a design concept. It's a just mean triggering a function when you've done something

So what is the core pieces of a training loop
- Fit
- Epoch
- Batch

We will wrap through each event here a `before` and `after` method with full access to the Learner (which includes everything: the model, optimizer, dataloader, ... ). `before_batch` `after_batch` `before_epoch` `after_epoch` `before_fit` `after_fit`

```python
class Learner():
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func=optim.SGD): fc.store_attr()

    def one_batch(self):
        self.preds = self.model(self.batch[0])
        self.loss = self.loss_func(self.preds, self.batch[1])
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()

    def one_epoch(self, train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        try:
            self.callback('before_epoch')
            for self.iter,self.batch in enumerate(self.dl):
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException: pass
            self.callback('after_epoch')
        except CancelEpochException: pass
    
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try:
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException: pass

    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)
```

The `exception` here is for adding more control / exitting an event when we need

Pause for a second and imagine if you want to write a DeviceCallBack, how do you do? ...

It's putting to the device:
- The model before fitting
- The input before each batch

To define a this Learner: ``` learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[DeviceCB(), metrics]) ```

How to go even further?

We don't want to repeat ourselves right? The `before` and `after` is repeating too much here. To solve this, we can use @decorator or @contextmanager for wrapping something before and after the event.

The fastai course experiment this idea further with even considering ```'predict','get_loss','backward','step','zero_grad'``` as events. It means there are really nothing that we can not control here. An example of when it is useful is experimenting a Momentum Learner by rewritting the `zere_grad`

```python
#|export
class MomentumLearner(TrainLearner):
    def __init__(self, model, dls, loss_func, lr=None, cbs=None, opt_func=optim.SGD, mom=0.85):
        self.mom = mom
        super().__init__(model, dls, loss_func, lr, cbs, opt_func)

    def zero_grad(self):
        with torch.no_grad():
            for p in self.model.parameters(): p.grad *= self.mom
     
```

It means instead of forgetting all the previous gradient by asigning it to zero, we can multiply it by a number < 1. So in the next update, we also take into account what we've trained before ( or momentum ). I don't know but maybe we shoud rename the method from `zero_grad` to something less confusing here.